In [240]:
# !apt-get update
# !apt-get install wkhtmltopdf pandoc libfontconfig1 libxrender1 libxext6
# !apt --fix-broken install

In [55]:
%pip install OpenAI markdown mdx_truly_sane_lists weasyprint python-dotenv pango

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for pango: filename=pango-0.0.1-py3-none-any.whl size=1041 sha256=feed0b1bc395a4ad8472f81191619fd6c052aafaa21e8d06cbef2fd6e5cd592c
  Stored in directory: /Users/andrewroberts/Library/Caches/pip/wheels/eb/63/bb/5483b38364b67a6781acf5ca46e3043f0551c78fa6de952a2d
Successfully built pango
Note: you may need to restart the kernel to use updated packages.


In [2]:
import os

from dotenv import load_dotenv
load_dotenv()  # Load the environment variables from .env file

# for Google Colab
#from google.colab import userdata
#os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')

#print(os.environ['OPENAI_API_KEY'])

True

# Setup

In [3]:
from openai import OpenAI

client = OpenAI()

# retrieve the Assistant
my_assistant = client.beta.assistants.retrieve("asst_3WQiw8VJ91XWikievImBqwrZ")

# create a thread
my_thread = client.beta.threads.create()


In [4]:
import time

def create_run_and_display_response(client, my_thread, my_assistant, content, file_ids=None):
    """
    Create a message, run it, and display the response.

    Args:
    client (Client): The API client.
    my_thread (Thread): The thread to post the message to.
    my_assistant (Assistant): The assistant to use for the run.
    content (str): The content of the message.
    file_ids (list, optional): List of file IDs to attach to the message.

    Returns:
    str: The response content.

    Raises:
    ValueError: If any of the parameters are invalid.
    RuntimeError: If the run does not complete successfully.
    """

    if not all([client, my_thread, my_assistant, content]):
        raise ValueError("Invalid parameters provided.")

    extra_instructions = """ ONLY return the content as markdown (with no delimiters) and with no commentary what so ever."""

    # Create the message
    thread_message = client.beta.threads.messages.create(
        my_thread.id,
        role="user",
        content=content + extra_instructions,
        file_ids=file_ids or []
    )

    # Hard code the values for max_retries and wait_seconds
    max_retries = 120
    wait_seconds = 0.5

    # Create a run
    run = client.beta.threads.runs.create(
        thread_id=my_thread.id,
        assistant_id=my_assistant.id
    )

    # Check the run's status
    for _ in range(max_retries):
        run = client.beta.threads.runs.retrieve(
            thread_id=my_thread.id,
            run_id=run.id)
        if run.status == "completed":
            break
        time.sleep(wait_seconds)
    else:
        raise RuntimeError("Run did not complete successfully.")

    # Retrieve and return the response
    all_messages = client.beta.threads.messages.list(thread_id=my_thread.id)
    return all_messages.data[0].content[0].text.value + "\n\n"



# Initial File Upload and Prompt

In [11]:
# get the job description from GitHub
import time


file_path = 'temp_files/jobdescription.txt'
# infra engineer example
#!wget -O file_path https://raw.githubusercontent.com/androb/deliverables/main/interviewplaybook/examples/infra-engineer/job-description.md
# medical assistant example
!wget -O 'temp_files/jobdescription.txt' https://raw.githubusercontent.com/androb/deliverables/main/interviewplaybook/examples/medical-assistant/job-description.md

# load it into a variable for later use
with open(file_path, 'r') as file:
    job_description = file.read()

# infra engineer example
# !wget -O companyvalues.txt https://raw.githubusercontent.com/androb/deliverables/main/interviewplaybook/examples/infra-engineer/company-values.md
# medical assistant example
!wget -O 'temp_files/companyvalues.txt' https://raw.githubusercontent.com/androb/deliverables/main/interviewplaybook/examples/medical-assistant/about-company.md
# time.sleep(5)


# Upload the file
file_company_values = client.files.create(
  file=open("temp_files/companyvalues.txt", "rb"),
  purpose="assistants"
)

# Upload the file
file_job_description = client.files.create(
  file=open("temp_files/jobdescription.txt", "rb"),
  purpose="assistants"
)
time.sleep(5)



--2023-12-19 16:53:47--  https://raw.githubusercontent.com/androb/deliverables/main/interviewplaybook/examples/medical-assistant/job-description.md
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2483 (2.4K) [text/plain]
Saving to: ‘temp_files/jobdescription.txt’

temp_files/jobdescr 100%[===================>]   2.42K  --.-KB/s    in 0s      

2023-12-19 16:53:47 (22.6 MB/s) - ‘temp_files/jobdescription.txt’ saved [2483/2483]

--2023-12-19 16:53:47--  https://raw.githubusercontent.com/androb/deliverables/main/interviewplaybook/examples/medical-assistant/about-company.md
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|1

# Priming the Pump

In [13]:
# Refresh the context to ensure it has read the files
pump_primer = create_run_and_display_response(client,my_thread,my_assistant,"""
            Write a detailed description of the company values from the company values file, using the company name.
            Then, write the job title contained in the job description file.
            Then, create the list of technical competencies, which MUST cover the areas mentioned in the
            requirements in the job description (and no more). Create the list of behavioral
            competencies, with a focus on the company values but still covering all requirements in the
            job description. Where possible, describe a competency in a way that aligns with the company values.
            Each competency should have a title followed by a short description.
            """, [file_job_description.id, file_company_values.id])
print(pump_primer)


Stanford Health Care values:
- Innovation: Embracing change and continuous improvement to raise the quality of care and shape the future of medicine.
- Connection: Cultivating community and care to nurture trust, respect, belonging, and shared purpose.
- Teamwork: Collaborating across roles and departments to unlock possibilities for greater impact.
- Mentorship: Nurturing professional growth for impactful and fulfilling careers.
- Participation: Valuing diverse contributions to improve decision-making and engagement.
- Action: Uniting perspectives for the greater good.

Job Title: Medical Assistant II

---

### Technical Competencies:

1. Clinical Proficiency
   Clinical skills and applying best medical practices in patient care, aligned with innovation in improving patient outcomes.

2. Patient Communication
   Effective communication techniques that demonstrate connection with patients, contributing to their well-being and care experience.

3. Computer Literacy
   Computer and softw

# Create Introduction

In [14]:
introduction = create_run_and_display_response(client,my_thread,my_assistant,"Generate the Introduction with the Candidate Profle (only if applicable) and Interview Structure. Be sure to include mention of the competencies in the interview structure.")
print(introduction)

# Interview Playbook for Medical Assistant II

## Introduction

Welcome to the Interview Playbook for Stanford Health Care's open position of Medical Assistant II. This document serves as a guide for interviewing teams to ensure a structured, comprehensive, and values-driven interview process. The role of Medical Assistant II is critical to the company's mission to provide the best patient care and experience. By following this guide, interviewers will be equipped to identify candidates who not only demonstrate required technical competencies but also embody the company's core values of Innovation, Connection, Teamwork, Mentorship, Participation, and Action. This will ensure the candidate's alignment with Stanford Health Care's dedication to excellence in healthcare.

### Interview Structure

- Greeting and Introduction (5-10 minutes): Build rapport while introducing the structure of the interview.
- Overview of Stanford Health Care and Role (5 minutes): Provide the candidate with info

# Create Evaluation Criteria

In [15]:
eval_criteria = create_run_and_display_response(client,my_thread,my_assistant,"""
    Generate the Evaluation Critera section.
    """)
print(eval_criteria)

## Evaluation Criteria

The following job scorecard is designed to evaluate candidates systematically based on the key competencies required for the Medical Assistant II position at Stanford Health Care. The scorecard assesses both technical and behavioral competencies that align with the company's values.

| Competency                   | Assessment | Comments        |
|------------------------------|------------|-----------------|
| Clinical Proficiency         |            |                 |
| Patient Communication        |            |                 |
| Computer Literacy            |            |                 |
| Medical Knowledge            |            |                 |
| Compliance and Ethics        |            |                 |
| Team Collaboration           |            |                 |
| Patient-Centered Care        |            |                 |
| Continuous Learning          |            |                 |
| Diversity and Inclusion      |            |      

# Create Technical Assessment

In [16]:
technical_assessment_1 = create_run_and_display_response(client,my_thread,my_assistant,"Generate the first Technical Assessment")
print(technical_assessment_1)


## Technical Assessment

The Technical Assessment portion of the interview is designed to thoroughly evaluate the candidate's capabilities that are directly relevant to the Medical Assistant II position at Stanford Health Care. It ensures that the candidate possesses the technical knowledge and practical skills required to excel in this role.

### Technical Competence 1: Clinical Proficiency

_Candidates must demonstrate a strong ability to perform a wide range of clinical tasks efficiently and effectively, adhering to Stanford Health Care's standards for patient care and safety. This includes patient assessments, assisting with procedures, managing patient portal messages, and maintaining a clean and organized work environment._

#### Lead Question

Describe a time when you had to perform a complex clinical procedure under pressure. How did you ensure the procedure was completed correctly and efficiently?

#### Probing Questions

- What protocols do you follow when preparing for a pro

In [17]:
technical_assessment_2 = create_run_and_display_response(client,my_thread,my_assistant,"Generate the 2nd Technical Assessment. Don't include the main section header.")
print(technical_assessment_2)

### Technical Competence 2: Patient Communication

_Candidates must be adept at engaging with patients in a manner that is both empathetic and informative, ensuring that communication contributes positively to their overall care experience. Proficiency here extends beyond verbal interactions to include writing patient care notes and instructions in a clear and understandable way._

#### Lead Question

Tell me about a difficult conversation you had to have with a patient or their family. How did you handle it, and what was the outcome?

#### Probing Questions

- How do you ensure information is conveyed effectively to patients with different levels of healthcare literacy?
- What strategies do you use to calm anxious patients or defuse tense situations?
- Can you give an example of how you have tailored your communication style to meet a patient's specific needs?
- Describe how you follow up with patients to ensure they understand their care plan and next steps.

#### Scenario

A patient

In [18]:
technical_assessment_3 = create_run_and_display_response(client,my_thread,my_assistant,"Generate the 3rd Technical Assessment. Don't include the main section header.")
print(technical_assessment_3)

### Technical Competence 3: Computer Literacy

_Computer literacy is crucial for a Medical Assistant II at Stanford Health Care. Candidates must demonstrate a strong ability to navigate electronic medical records (EMR), utilize standard office software, and effectively manage digital communications with patients and the healthcare team._

#### Lead Question

Can you walk me through a complex task you’ve had to manage using an EMR system, and how you ensured accuracy and efficiency?

#### Probing Questions

- Describe your experience with scheduling and managing patient appointments within an EMR.
- How do you approach learning new medical software or technology tools?
- What steps do you take to ensure the confidentiality and security of patient information when using computers?
- Share an experience where you had to troubleshoot a technical issue within an EMR system. What was the outcome?

#### Scenario

You're in the middle of documenting patient health information in the EMR when t

In [19]:
technical_assessment_4 = create_run_and_display_response(client,my_thread,my_assistant,"Generate the 4th Technical Assessment. Don't include the main section header.")
print(technical_assessment_4)

### Technical Competence 4: Medical Knowledge

_Medical knowledge encompasses a thorough understanding of medical terminology, procedures, patient care, and healthcare compliance. For a Medical Assistant II at Stanford Health Care, it also means staying current with the latest healthcare guidelines and conveying that knowledge effectively for patient education._

#### Lead Question

Describe a scenario where your medical knowledge significantly impacted the outcome of a patient's care.

#### Probing Questions

- How do you ensure your understanding of medical terminology is comprehensive and up-to-date?
- Can you provide an example of how you have educated a patient using your medical knowledge?
- What is your approach to staying informed about changes in healthcare regulations and policies?
- Share a situation where you identified a potential health risk based on your medical knowledge and how you addressed it.

#### Scenario

You're presented with a patient who exhibits symptoms that

In [20]:
technical_assessment_5 = create_run_and_display_response(client,my_thread,my_assistant,"Generate the 5th Technical Assessment. Don't include the main section header.")
print(technical_assessment_5)

### Technical Competence 5: Compliance and Ethics

_Adhering to healthcare laws, regulations, and ethical standards is integral to the role of a Medical Assistant II at Stanford Health Care. Candidates must demonstrate an understanding of these frameworks and the ability to apply them to everyday clinical operations._

#### Lead Question

Give an example of how you have handled a situation where patient care practices were in question with respect to compliance or ethics.

#### Probing Questions

- Describe the steps you take to ensure compliance with HIPAA and other patient privacy laws in your day-to-day responsibilities.
- What would you do if you noticed a coworker violating patient confidentiality or engaging in unethical conduct?
- Can you discuss a time when you had to navigate a grey area in healthcare compliance and how you made your decision?
- How do you maintain up-to-date knowledge of healthcare regulations that affect your role?

#### Scenario

You are assisting with a pr

In [21]:
technical_assessment = technical_assessment_1 + technical_assessment_2 + technical_assessment_3 + technical_assessment_4 + technical_assessment_5

# Create Behavioral Assessment

In [22]:
behavioral_assessment_1 = create_run_and_display_response(client,my_thread,my_assistant,"Generate the 1st Behavioral Assessment section.")
print(behavioral_assessment_1)

## Behavioral Assessment

The Behavioral Assessment part of the interview focuses on evaluating the candidate's soft skills and how well they align with Stanford Health Care's values. This allows the interviewer to determine the candidate's potential for teamwork, personal growth, and adherence to the company culture.

### Behavioral Competence 1: Team Collaboration

_A Medical Assistant II at Stanford Health Care must demonstrate the ability to work seamlessly with team members across various departments. This competency reflects the company's value of Teamwork, ensuring collaborative and high-impact healthcare delivery._

#### Lead Question

Tell me about a project or situation where you had to work closely with a team to achieve a health care goal. What was your role, and how did the group dynamic contribute to the outcome?

#### Probing Questions

- How do you navigate challenges when working with diverse healthcare teams?
- Can you give an example of when you helped resolve a conf

In [23]:
behavioral_assessment_2 = create_run_and_display_response(client,my_thread,my_assistant,"Generate the 2nd Behavioral Assessment section. Don't include the main section header.")
print(behavioral_assessment_2)

### Behavioral Competence 2: Patient-Centered Care

_Patient-Centered Care at Stanford Health Care means prioritizing the patient's emotional, mental, and physical well-being throughout their healthcare experience. Candidates are expected to exhibit behaviors that align with the value of Connection, creating a supportive and empathetic care environment._

#### Lead Question

Can you share an experience where you went above and beyond to meet a patient's needs? What impact did this have on their care experience?

#### Probing Questions

- How do you tailor your approach to care based on individual patient needs and situations?
- Describe a time when you had to advocate for a patient. What steps did you take and what was the result?
- What strategies do you use to ensure patients feel heard and supported during their visit?
- Can you provide an example of how you keep patients’ cultural and personal preferences in mind during care?

#### Scenario

You encounter a patient who is extremely

In [24]:
behavioral_assessment_3 = create_run_and_display_response(client,my_thread,my_assistant,"Generate the 3rd Behavioral Assessment section. Don't include the main section header.")
print(behavioral_assessment_3)

### Behavioral Competence 3: Continuous Learning

_Stanford Health Care values a culture of continuous learning and improvement, aligning with Innovation and Mentorship. Candidates for Medical Assistant II should demonstrate a dedication to personal and professional development._

#### Lead Question

Reflect on a time when you had to learn a new skill or procedure to improve your work performance. How did you go about mastering it?

#### Probing Questions

- How do you identify areas for personal growth and seek opportunities to enhance your skills?
- Describe a learning challenge you faced in your career and how you overcame it.
- Can you give an example of how you've shared knowledge with your colleagues to help elevate team competence?
- What resources or strategies do you use to stay updated with the latest medical practices?

#### Scenario

Assume you are introduced to a new piece of medical technology that is set to become standard in your facility. Outline the steps you would ta

In [25]:
behavioral_assessment_4 = create_run_and_display_response(client,my_thread,my_assistant,"Generate the 4th Behavioral Assessment section. Don't include the main section header.")
print(behavioral_assessment_4)

### Behavioral Competence 4: Diversity and Inclusion

_Stanford Health Care is committed to fostering a diverse and inclusive environment, where all staff and patients feel valued and able to contribute. This behavioral competency aligns with the values of Connection and Participation._

#### Lead Question

Share a time when you worked with a diverse group of colleagues or patients and how you navigated any differences to ensure effective collaboration and care.

#### Probing Questions

- How do you adapt your communication style to connect with individuals from various backgrounds?
- Can you provide an example of how you have contributed to creating an inclusive workplace?
- What have you learned from working with people who have different perspectives from your own?
- Describe a situation where you recognized unconscious bias, either in yourself or others, and how you addressed it.

#### Scenario

Imagine a scenario where language barriers with a patient are leading to miscommunicati

In [26]:
behavioral_assessment_5 = create_run_and_display_response(client,my_thread,my_assistant,"Generate the 5th Behavioral Assessment section. Don't include the main section header.")
print(behavioral_assessment_5)

### Behavioral Competence 5: Proactive Initiative

_Stanford Health Care encourages a proactive approach to improving processes and patient care, which is anchored in the company values of Innovation and Action. Medical Assistant II candidates should exemplify the ability to take initiative in enhancing their workplace._

#### Lead Question

Can you discuss a time when you identified a potential improvement in your work environment and took the lead on implementing it?

#### Probing Questions

- How do you stay motivated to continuously seek improvements in your role?
- Describe how you approach problem-solving when you see an opportunity for process enhancement.
- What steps do you take to ensure that any changes you propose are aligned with both team goals and patient care standards?
- Can you give an example of a time when your proactive actions directly benefited patient safety or experience?

#### Scenario

You notice that the current system for managing patient follow-ups is inef

In [27]:
behavioral_assessment = behavioral_assessment_1 + behavioral_assessment_2 + behavioral_assessment_3 + behavioral_assessment_4 + behavioral_assessment_5

# Create Practical Assignment

In [28]:
practical_assignment = create_run_and_display_response(client,my_thread,my_assistant,"Generate the Practical Assignment section.")
print(practical_assignment)


## Practical Assignment

The practical assignment is an essential component of the interview process for the Medical Assistant II position at Stanford Health Care. It is designed to evaluate the candidate's hands-on skills and problem-solving abilities in a controlled, real-world scenario that reflects the demands of the role.

### Practical Assignment Description

The candidate will be asked to perform a set of clinical tasks relevant to the daily responsibilities of a Medical Assistant II. This may include a patient check-in process, basic diagnostic procedures, EMR documentation, and a patient education session on a common healthcare topic.

### Objectives

- Assess the candidate's proficiency in performing core medical assistant duties.
- Evaluate the candidate's ability to utilize medical equipment and technology effectively.
- Gauge the candidate's communication skills and patient interaction techniques.
- Observe the candidate's organizational skills and ability to prioritize ta

# Create Team Interaction

In [29]:
team_interaction = create_run_and_display_response(client,my_thread,my_assistant,"Generate the Team Interaction section.")
print(team_interaction)

## Team Fit

Team interaction is a vital component of the interview process for the Medical Assistant II position at Stanford Health Care. It helps assess how well the candidate will mesh with the existing team and adapt to the company culture, which is built upon values such as teamwork, connection, and participation.

### Rationale for Team Involvement

Involving the team in the interview process provides valuable insight into the candidate's ability to communicate, collaborate, and contribute to a positive and productive work environment. It allows current team members to gauge the candidate's potential fit within the team dynamic and to envision their future working relationship.

### Potential Discussion Points

- The candidate's experience working in diverse healthcare teams.
- Specific instances where the candidate has contributed to team success or confronted challenges.
- The candidate's approach to sharing feedback and receiving constructive criticism.
- The candidate's ideas

# Create Candidate Q&A

In [30]:
candidate_questions = create_run_and_display_response(client,my_thread,my_assistant,"""
    Generate the Candidate Q&A section. Include example answers.
    """)
print(candidate_questions)

## Candidate Q&A

After the formal interview, it is typical for candidates to have questions of their own. This provides an opportunity for them to better understand the position at Stanford Health Care and the company as a whole. Here are example questions candidates might ask, along with comprehensive answers.

### Example Candidate Questions:

1. What does a typical day look like for a Medical Assistant II at Stanford Health Care?
   
   _Answer: A typical day involves varied patient interactions, including check-ins, assisting with clinical procedures, updating patient records, and interfacing with other healthcare professionals. You'll be expected to balance direct patient care with administrative tasks in our electronic medical record system._

2. How does Stanford Health Care support continuous learning and professional development?
   
   _Answer: We offer a variety of professional development opportunities, including continuing education courses, seminars, mentorship programs,

# Finalize the Playbook HTML

## Create a markdown version

For use in GitHub as an example

In [31]:
playbook_md = (introduction + "\n\n" +
            job_description + "\n\n" +
            eval_criteria + "\n\n" +
            technical_assessment + "\n\n" +
            behavioral_assessment + "\n\n" +
            practical_assignment + "\n\n" +
            candidate_questions + "\n\n" +
            team_interaction + "\n\n" )

# Write playbook to a text file
with open('playbook.md', 'w') as file:
    file.write(playbook_md)

## Convert the markdown to HTML

We are doing this in chunks so that we can do more useful things with it in the HTML output

In [32]:
import markdown

# Convert Markdown to HTML
html_playbook = markdown.markdown(playbook_md, extensions=['extra','mdx_truly_sane_lists'])

# Convert Markdown to HTML
html_job_description = markdown.markdown(job_description, extensions=['extra','mdx_truly_sane_lists'])
html_introduction = markdown.markdown(introduction, extensions=['extra','mdx_truly_sane_lists'])
html_technical_assessment = markdown.markdown(technical_assessment, extensions=['extra','mdx_truly_sane_lists'])
html_behavioral_assessment = markdown.markdown(behavioral_assessment, extensions=['extra','mdx_truly_sane_lists'])
html_practical_assignment = markdown.markdown(practical_assignment, extensions=['extra','mdx_truly_sane_lists'])
html_candidate_questions = markdown.markdown(candidate_questions, extensions=['extra','mdx_truly_sane_lists'])
html_team_interaction = markdown.markdown(team_interaction, extensions=['extra','mdx_truly_sane_lists'])
html_eval_criteria = markdown.markdown(eval_criteria, extensions=['extra','mdx_truly_sane_lists'])


html_technical_assessment_1 = markdown.markdown(technical_assessment_1, extensions=['extra','mdx_truly_sane_lists'])
html_technical_assessment_2 = markdown.markdown(technical_assessment_2, extensions=['extra','mdx_truly_sane_lists'])
html_technical_assessment_3 = markdown.markdown(technical_assessment_3, extensions=['extra','mdx_truly_sane_lists'])
html_technical_assessment_4 = markdown.markdown(technical_assessment_4, extensions=['extra','mdx_truly_sane_lists'])
html_technical_assessment_5 = markdown.markdown(technical_assessment_5, extensions=['extra','mdx_truly_sane_lists'])

html_behavioral_assessment_1 = markdown.markdown(behavioral_assessment_1, extensions=['extra','mdx_truly_sane_lists'])
html_behavioral_assessment_2 = markdown.markdown(behavioral_assessment_2, extensions=['extra','mdx_truly_sane_lists'])
html_behavioral_assessment_3 = markdown.markdown(behavioral_assessment_3, extensions=['extra','mdx_truly_sane_lists'])
html_behavioral_assessment_4 = markdown.markdown(behavioral_assessment_4, extensions=['extra','mdx_truly_sane_lists'])
html_behavioral_assessment_5 = markdown.markdown(behavioral_assessment_5, extensions=['extra','mdx_truly_sane_lists'])



 ## Create the style for the document

Borrowed colors from this [color palette](https://color.adobe.com/PALETTE-mini-06:-Transparent-color-theme-15d8c4a6-b06a-4530-bf5b-3fc8102ccd69)

In [39]:
styles= """

   body {
        font-family: Source Sans Pro;
        font-size: 11pt;
        font-weight: 200;
        font-stretch: 75% 125%;
        color:#0D0D0D;
    }
    @page {
        margin: 0.65in;
        size: Letter;
        @bottom-right {
          content: counter(page);
          font-size: 11pt;
          color: #73706C;
        }
    }
    @page :first {
        @bottom-right {
          content: none;
        }
    }


    h1, h2, h3, h4 {
        font-family: Libre Baskerville;
        margin-top: 1em;
        margin-bottom: 0.5em;
    }
    h1 {
        font-size: 24pt;
        margin-top: 3em;
        page-break-after: avoid;
    }
    .intro h1{
        font-size: 50pt;
        margin-top: 3em;
        page-break-after: always;
    }
    @media print {
      /* Define styles for the cover page */
      .title {
        position: fixed;
        left: 0;
        top: 0;
        width: 100%;
        height: 100%;
        background-color: #F2F2F2; /* Change as needed */
        color: #0D0D0D; /* Change as needed */
        text-align: center;
        padding: 50px;
      }
    }
    .publication-date {
      position: absolute;
      bottom: 10px; /* adjust as needed */
      width: 100%;
      text-align: left;
      font-size: 11pt; /* adjust as needed */
      color: #73706C; /* adjust as needed */
    }


    h2 {
        font-size: 18pt;
        page-break-after: avoid;
    }
    h3 {
        font-size: 14pt;
        page-break-after: avoid;
    }
    h4 {
        font-size: 12pt;
        page-break-after: avoid;
    }
    p {
        margin-bottom: 1em;
    }


    h1, h2, h3, h4 {
        page-break-after: avoid;
    }

    p, table, figure {
        page-break-inside: avoid;
    }
    table {
        width: 100%;
        border-collapse: collapse;
        margin-bottom: 1em;
        page-break-before: avoid;
    }
    th, td {
        border: 1px solid #ddd;
        padding: 8px;
        text-align: left;
        font-size: 10pt;
    }
    th {
        background-color: #f2f2f2;
        font-weight: bold;
    }
    tr:nth-child(even) {{background-color: #f9f9f9;}}

    ul, ol {
        margin-bottom: 1em;
        margin-left: 0;
        padding-left:1.5em;
    }
    li {
        margin-bottom: 0.5em;
        margin-left: 0em;
    }

    .main {
      page-break-before: always;
    }

    .dotted-line {
        border-bottom: 1px dotted black;
        width: 100%;
        margin-top: 20px; /* This ensures spacing between each line */
        margin-bottom: 20px; /* Adjust if needed */
    }

    @media print {
        .dotted-line {
            margin-top: 20px; /* Adjust as needed for printing */
            margin-bottom: 20px; /* Adjust as needed for printing */
        }
    }

    .rating-container {
        display: flex;
        align-items: flex-end;
        margin-top: 20px;
        margin-bottom: 20px;
    }

    .rating-container label {
        margin-right: 10px;
        font-size: 10pt;
        font-family: Source Sans Pro;
    }

    .short-dotted-line {
        border-bottom: 1px dotted black;
        width: 150px; /* Adjust the width as needed */
        margin-left: 5px;
    }}

    @media print {
        .short-dotted-line {
            width: 150px; /* Adjust for printing if needed */
        }
    }
"""

## Create some useful HTML snippets

In [45]:

html_notes = """<h4>Notes</h4><div class='notes'>
    <div class="dotted-line"></div>
    <div class="dotted-line"></div>
    <div class="dotted-line"></div>
    <div class="dotted-line"></div>
    <div class="dotted-line"></div>
    <div class="dotted-line"></div>
    <div class="dotted-line"></div>
    <div class="rating-container">
    <label for="rating-line">Rating (1-5):</label>
    <div id="rating-line" class="short-dotted-line"></div>
    </div>
</div>"""

html_short_notes = """<h4>Notes</h4><div class='notes'>
    <div class="dotted-line"></div>
    <div class="dotted-line"></div>
    <div class="dotted-line"></div>
    <div class="dotted-line"></div>
    <div class="rating-container">
    <label for="rating-line">Rating (1-5):</label>
    <div class="short-dotted-line"></div>
    </div>
</div>"""

from datetime import datetime

# Getting today's date in a nicely formatted string for US English
publication_date = datetime.now().strftime("%B %d, %Y")

## Build the HTML

In [41]:
# Define the HTML template
html_content = rf"""
<meta charset="utf-8">
<html>
<head>
  <link href="https://fonts.googleapis.com/css2?family=Source%20Sans%20Pro:wght@400;700" rel="stylesheet">
  <link href="https://fonts.googleapis.com/css2?family=Libre%20Baskerville:wght@400;700" rel="stylesheet">
  <style>
  {styles}
  </style>
</head>
<body>
    <section class='intro'>
    <div class="publication-date">Published on {publication_date} by the <a href="https://github.com/androb/deliverables/tree/main/interviewplaybook">Interview Playbook Generator</a></div>
    {html_introduction}
    </section>
    <section class='main'>
    {html_job_description}
    </section>
    <section class='main'>
    {html_eval_criteria}
    </section>
    <section class='main'>
      {html_technical_assessment_1}
     {html_short_notes}
    </section>
    <section class='main'>
      {html_technical_assessment_2}
     {html_notes}
    </section>
        <section class='main'>
      {html_technical_assessment_3}
     {html_notes}
    </section>
        <section class='main'>
      {html_technical_assessment_4}
     {html_notes}
    </section>
        <section class='main'>
      {html_technical_assessment_5}
     {html_notes}
    </section>
    <section class='main'>
    {html_behavioral_assessment_1}
     {html_short_notes}
    </section>
    <section class='main'>
    {html_behavioral_assessment_2}
     {html_notes}
    </section>
        <section class='main'>
    {html_behavioral_assessment_3}
     {html_notes}
    </section>
        <section class='main'>
    {html_behavioral_assessment_4}
     {html_notes}
    </section>
        <section class='main'>
    {html_behavioral_assessment_5}
     {html_notes}
    </section>
    <section class='main'>
    {html_practical_assignment}
    {html_notes}
    </section>
    <section class='main'>
    {html_team_interaction}
    {html_notes}
    </section>
    <section class='main'>
    {html_candidate_questions}
    </section>

</body>
</html>"""

# Write HTML string to a file
with open('playbook.html', 'w') as file:
    file.write(html_content)



Display the HTML content here so we can review it

In [42]:
# Display the Markdown-formatted text in the notebook
from IPython.display import display, HTML
display(HTML(html_content))

Competency,Assessment,Comments
Clinical Proficiency,,
Patient Communication,,
Computer Literacy,,
Medical Knowledge,,
Compliance and Ethics,,
Team Collaboration,,
Patient-Centered Care,,
Continuous Learning,,
Diversity and Inclusion,,
Proactive Initiative,,


# Create the Documents

Create the Word document

In [47]:
# Write HTML string to a file
with open('playbook.html', 'w') as file:
    file.write(html_content)

# Convert HTML to Docx
!pandoc -s playbook.html -o playbook.docx

# from google.colab import files
# files.download('playbook.docx')

Create the PDF

In [1]:
from weasyprint import HTML

# Convert HTML to PDF using WeasyPrint
#HTML('playbook.html').write_pdf('playbook.pdf', stylesheets=['style.css'], presentational_hints=True)
HTML('playbook.html').write_pdf('playbook.pdf')

# from google.colab import files
# files.download('playbook.pdf')

# Convert HTML to alternative PDF
#!pandoc -s playbook.html --pdf-engine=wkhtmltopdf -o playbook.pdf

# Convert HTML to alternative PDF
#!wkhtmltopdf --page-size Letter --footer-html footer.html --margin-top 15mm --margin-bottom 15mm --margin-left 15mm --margin-right 15mm playbook.html playbook.pdf

# Convert HTML to PDF
##import pdfkit
# pdfkit.from_string(html_content, 'playbook.pdf')



-----

WeasyPrint could not import some external libraries. Please carefully follow the installation steps before reporting an issue:
https://doc.courtbouillon.org/weasyprint/stable/first_steps.html#installation
https://doc.courtbouillon.org/weasyprint/stable/first_steps.html#troubleshooting 

-----



OSError: cannot load library 'pango-1.0-0': dlopen(pango-1.0-0, 0x0002): tried: 'pango-1.0-0' (no such file), '/System/Volumes/Preboot/Cryptexes/OSpango-1.0-0' (no such file), '/Users/andrewroberts/anaconda3/lib/python3.11/site-packages/../../pango-1.0-0' (no such file), '/Users/andrewroberts/anaconda3/bin/../lib/pango-1.0-0' (no such file), '/usr/lib/pango-1.0-0' (no such file, not in dyld cache), 'pango-1.0-0' (no such file), '/usr/local/lib/pango-1.0-0' (no such file), '/usr/lib/pango-1.0-0' (no such file, not in dyld cache).  Additionally, ctypes.util.find_library() did not manage to locate a library called 'pango-1.0-0'